<a href="https://colab.research.google.com/github/FforZakhar/DZ-matstat-/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json
import nltk.stem
import string
from nltk.corpus import stopwords
import re

pd.options.mode.chained_assignment = None # Fuck warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Kaggle case/train.csv')

# Checking out

In [ ]:
df.iloc[19655]

Id                            922a5f2c-2d1c-46a7-a07a-acaf2222c0c6
pub_title        3D U-Net for segmentation of COVID-19 associat...
dataset_title    RSNA International COVID-19 Open Radiology Dat...
dataset_label    RSNA International COVID-19 Open Radiology Dat...
cleaned_label    rsna international covid 19 open radiology dat...
Name: 19655, dtype: object

In [ ]:
df.iloc[-1] == df.iloc[-2]

Id                True
pub_title         True
dataset_title     True
dataset_label    False
cleaned_label    False
dtype: bool

In [ ]:
df.iloc[-2].dataset_label

'CAS COVID-19 antiviral candidate compounds data set'

In [ ]:
df.iloc[-1].dataset_label

'CAS COVID-19 antiviral candidate compounds data'

In [ ]:
df.columns

Index(['Id', 'pub_title', 'dataset_title', 'dataset_label', 'cleaned_label'], dtype='object')

In [ ]:
df.Id

0        d0fa7568-7d8e-4db9-870f-f9c6f668c17b
1        2f26f645-3dec-485d-b68d-f013c9e05e60
2        c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29
3        5c9a3bc9-41ba-4574-ad71-e25c1442c8af
4        c754dec7-c5a3-4337-9892-c02158475064
                         ...                 
19656    b3498176-8832-4033-aea6-b5ea85ea04c4
19657    f77eb51f-c3ac-420b-9586-cb187849c321
19658    ab59bcdd-7b7c-4107-93f5-0ccaf749236c
19659    fd23e7e0-a5d2-4f98-992d-9209c85153bb
19660    fd23e7e0-a5d2-4f98-992d-9209c85153bb
Name: Id, Length: 19661, dtype: object

# Useful functions

In [ ]:
# Getting info from the publication json

def getPublData(id):
  with open(f'/content/drive/MyDrive/Kaggle case/train/{id}.json', 'r') as f:
    raw_data = json.load(f)
  
  data = [(part["section_title"], part['text']) for part in raw_data] # New data format is list of tuples: (section_name, section_text)
  # for part in raw_data:
  #   data.append((part["section_title"], part['text']))
  
  return data

In [ ]:
# Defining stem

my_stem = nltk.stem.SnowballStemmer('english') # Maybe it`s not the best choice[*]

def stemSentence(sentence):
    new_sentence = ''

    for word in sentence.split():
        new_sentence += my_stem.stem(word) + ' '
    
    return new_sentence[:-1]

In [ ]:
# Deleting stop words function. If it`s useful could be merged with previous one
# But initially it`s necessary to check that there aren`t words from data sets title in stopwords [*]

def deleteStopWords(sentence):
    res = ''
    
    for word in sentence.split():
        if word not in stopwords.words('english'):
            res += word + ' '
    
    return res[:-1]

In [ ]:
# Agregate text editting function

def editText(text):
  text = ''.join([char.lower() for char in text if char not in string.punctuation])
  text = stemSentence(text)
  text = deleteStopWords(text)
  return text

In [ ]:
# Func editing publdata structures

def editPubldata(publdata):
  return tuple([(editText(datum[0]), editText(datum[1])) for datum in publdata])

In [ ]:
# Some code from stackoverflow allowing to multiprocess pandas apply function
# https://stackoverflow.com/questions/26784164/pandas-multiprocessing-apply

from multiprocessing import  Pool
from functools import partial


def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [ ]:
# Official text cleaning function

import regex as re


def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower())

# Preprocessing

In [ ]:
df = df.rename(columns={'Id': 'id'})  # Lowercase the columns names

In [ ]:
# Creating new column with publication text data

df['publdata'] = parallelize_on_rows(df.id, getPublData, num_of_processes=256)

In [ ]:
# Editting text in publdata

df['fixed_publdata'] = parallelize_on_rows(df.publdata, editPubldata, num_of_processes=256)

In [ ]:
# It should be deleted in final version
# Saving preprocessed dataset

df.to_json('/content/drive/MyDrive/Kaggle case/preprocessed_dataset.json')

In [ ]:
# It should be deleted in final version
# Loading preprocessed dataset

df = pd.read_json('/content/drive/MyDrive/Kaggle case/preprocessed_dataset.json')

# Data analysis

In [ ]:
def getContext(key_words, text):
  result = []
  text_len = len(text)
  key_words_len = len(key_words)

  i = text.find(key_words)
  while i != -1:
    start = i - 50
    end = i + key_words_len + 51

    if start < 0:
      start = 0
    if end > text_len:
      end = text_len
    words_to_append = text[start:end]
    words_to_append = words_to_append.replace(key_words, ' ')
    result.append((words_to_append.split()))

    text = text[i+1:]
    i = text.find(key_words)
  
  return result

In [ ]:
df.iloc[0].new_publdata.split(',')[1][2:-2]

'this studi use data from the nation educ longitudin studi nels88 to examin the effect of dual enrol program for high school student on colleg degre attain the studi also report whether the impact of dual enrol program were differ for first generat colleg student versus student whose parent had attend at least some colleg in addit a supplement analysi report on the impact of differ amount of dual enrol coursetak and colleg degre attain dual enrol program offer collegelevel learn experi for high school student the program offer colleg cours andor the opportun to earn colleg credit for student while still in high school the intervent group in the studi was compris of nel particip who attend a postsecondari school and who particip in a dual enrol program while in high school n 880 the studi author use propens score match method to creat a comparison group of nel particip who also attend a postsecondari school but who did not particip in a dual enrol program in high school n 7920'